This notebook computes the derivative of the resilience indicator. This information is usefull for rendering the scorcards. This notebook must be run after compute_res_ind and before render_scorecards

In [3]:
%reset-f
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from res_ind_lib import *

# data

In [4]:
#Load the excel data file into a dataframe (table)
df_original=pd.read_excel("inputs/all_data_compiled.xlsx", 
                 index_col=0, #tells Pandas to use the first column (Provinces) as the index of the table
                 skiprows=[0, 2] #The first (0) row is a description of the variables and the 3rd (2) row is empty. We skip them
                ) 
df_original.index.name="province"
#Displays the first rows of that dataframe
df_original=def_ref_values(df_original)

# computes all derivatives

In [5]:
deriv_set = np.setdiff1d( df_original.columns,
    ["pop","iso3","original_country","income_elast","avg_prod_k","axfin_p","axfin_r","v_s","gdp_pc_pp","gdp_pc_pp_nat",
     "protectionref","gdp_pc_pp_ref","rho", "T_rebuild_K" ])

pd.DataFrame(data=deriv_set).to_csv("inputs/deriv_set.csv",index=False,header=False)

pol_assess_set = ["dKtot","dWtot_currency"]
              
#Checks that info has information for all variables in deriv_set
info = pd.read_csv("inputs/inputs_info.csv").set_index("key")
for d in deriv_set:
    if d not in info.index.values:
        raise Exception(d+" is not documented in inputs_info.csv")

        


In [6]:
from progress_reporter import *

###MH set adds and compares

def compute_derivative(df_original,deriv_set,pol_assess_set, **kwargs):
    der = pd.DataFrame(index=df_original.index, columns=pd.MultiIndex.from_product([deriv_set,pol_assess_set], names=['inputs', 'outputs']))
    h=0.0001
    #loop on all data in df prior to add the results
    fx = compute_resiliences(df_original, **kwargs)[pol_assess_set]
    for var in deriv_set:
        try:
            progress_reporter(var)
            df_=df_original.copy(deep=True)
            df_[var]=df_[var]+h
            fxh= compute_resiliences(df_, **kwargs)[pol_assess_set]
            der[var] = ((fxh-fx)/(h))  # for elasticity: .mul(df_[var],axis=0)/fx
        except TypeError:
            print("no derivative for " +var)
    progress_reporter("done.")        
    return der.swaplevel('inputs', 'outputs', axis=1).sort_index(axis=1).dropna(how="all",axis=1)

In [7]:
fr=pd.read_csv("inputs/fa_ratios_river.csv", index_col="province").dropna()
fr.columns = fr.columns.astype(float)
fr.head()

,10.0,100.0
province,,
Abra,1,1.814138
Agusan Del Norte,1,1.081761
Agusan Del Sur,1,1.335951
Apayao,1,1.000000
Batangas,1,1.000000


In [8]:
derivatives = compute_derivative(df_original,deriv_set,pol_assess_set, fa_ratios=fr)
derivatives.head()

Currently working on:  done.


outputs                   dKtot                                              \
inputs                      fap            far    nat_buyout             pi   
province                                                                      
Abra               86515.763355  241586.279502 -7.275958e-08  -61758.704609   
Agusan Del Norte  527119.638492  990375.206298  5.820766e-07 -474279.819536   
Agusan Del Sur    681083.520832  702192.265787  0.000000e+00 -156300.480151   
Aklan                       NaN            NaN           NaN            NaN   
Albay                       NaN            NaN           NaN            NaN   

outputs                                                                      \
inputs                 pov_head    protection     rel_gdp_pp         share1   
province                                                                      
Abra               21006.658870  -4115.116595   69686.022313   15414.428731   
Agusan Del Norte  433636.047360 -50352.386635  463959.045559  408336.902522   
Agusan Del Sur    138517.292013 -17217.231692  216386.045422  127806.090948   
Aklan                       NaN           NaN            NaN            NaN   
Albay                       NaN           NaN            NaN            NaN   

outputs                                      ...       dWtot_currency  \
inputs                     shew sigma_p      ...           protection   
province                                     ...                        
Abra              -12955.246179       0      ...         -6280.186856   
Agusan Del Norte -115498.194395       0      ...        -66615.261146   
Agusan Del Sur    -38062.811228       0      ...        -31212.059781   
Aklan                       NaN     NaN      ...                  NaN   
Albay                       NaN     NaN      ...                  NaN   

outputs                                                                       \
inputs               rel_gdp_pp         share1           shew        sigma_p   
province                                                                       
Abra              -53097.054300  -40905.090912  -19920.908018  -12081.676970   
Agusan Del Norte -305247.180049 -636985.968724 -155973.862525 -316030.714214   
Agusan Del Sur   -195343.305326 -209075.758557  -70079.002447 -132153.490059   
Aklan                       NaN            NaN            NaN            NaN   
Albay                       NaN            NaN            NaN            NaN   

outputs                                                                   \
inputs               sigma_r       social_p     social_r             v_p   
province                                                                   
Abra             -193.290230  -20431.767805  -326.869333   223998.087511   
Agusan Del Norte -867.494416 -516391.740923 -1417.436702  1185765.828512   
Agusan Del Sur   -394.496972 -215937.788987  -644.584300   683525.016827   
Aklan                    NaN            NaN          NaN             NaN   
Albay                    NaN            NaN          NaN             NaN   

outputs                          
inputs                      v_r  
province                         
Abra              185509.497212  
Agusan Del Norte  910252.941618  
Agusan Del Sur    324630.736384  
Aklan                       NaN  
Albay                       NaN  

[5 rows x 30 columns]

# output

output signs in excel

In [9]:
#saves derivatives in excel tabs with signs in colors (very usefull for understanding the model)
writer= pd.ExcelWriter("results/derivatives.xlsx", engine='xlsxwriter')
workbook=writer.book
# Add a format. Light red fill with dark red text.
red = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

blue = workbook.add_format({'bg_color': '#92c5de',
                               'font_color': '#000061'})
for outname in pol_assess_set:
    sheetname = outname[0:30]
    (derivatives[outname].dropna()).to_excel(writer,sheet_name=sheetname)
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '>',
                                    'value':    0,
                                    'format':   blue})
    writer.sheets[sheetname].conditional_format('B2:BB600', {'type':'cell',
                                    'criteria': '<',
                                    'value':    0,
                                    'format':   red})
    writer.sheets[sheetname].freeze_panes(1, 1)


In [10]:
import warnings
warnings.filterwarnings("always",category=UserWarning)
try :
    writer.save()
except PermissionError:
    warnings.warn("Cannot write excel file. Check that it's not opened and try again")


In [11]:
## consistency check

In [12]:
for out in pol_assess_set:
    der =     np.sign(derivatives[out]).replace(0,np.nan)
    signs= pd.Series(index=der.columns)
    for i in signs.index:
        if (der[i].min()==der[i].max()): #all nonnan signs are equal
            signs[i]=der[i].min()
        else:
            print("ambigous sign for "+i+" on "+out)
            signs[i]=np.nan

ambigous sign for nat_buyout on dKtot
ambigous sign for sigma_p on dKtot
ambigous sign for sigma_r on dKtot
ambigous sign for social_p on dKtot
ambigous sign for social_r on dKtot
